# Distance Notebook - DEPRECIATED - CODE IS SPAGHETTI
## Refactored - impoved book in workbook folder (as of 10-2-24)
### Calculate the distance each team will need to travel over the course of the season

#### Note

In [44]:
# Dependencies
import os
import sys
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic



In [45]:
## Paths to source data

# schedule_path = os.path.join('..', 'data', 'schedule', '2024_current.csv')
schedule_path = os.path.join('..', 'data', 'schedule', 'CHN_Schedule_First Pass_v1.csv')
schedule_data = pd.read_csv(schedule_path)
raw_schedule_df = schedule_data.copy()

# ARENA INFO FILE
arena_path = os.path.join('..', 'data', 'arena_school_info.csv')
arena_data = pd.read_csv(arena_path)
arena_info_df = arena_data.copy()

# NEUTRAL SITE ARENA INFORMATION FILE
neutral_path = os.path.join('..', 'data', 'neutral_arenas_2024.csv')
neutral_arenas_df = pd.read_csv(neutral_path)

# Display data
# schedule_data.head()
# schedule_data.tail()
# arena_data.head()
# neutral_arenas_df.tail()

## NEW ATTEMPT WITH PAIRED DOWN LOGIC

In [46]:
# Complete refactored code block with corrections
## Ensure the Data column in the schedule data is in datetime format
raw_schedule_df['Date'] = pd.to_datetime(raw_schedule_df['Date'])

from geopy.distance import geodesic

# ## Remove ' from all Team columns
raw_schedule_df['Home_Team'] = raw_schedule_df['Home_Team'].str.replace("'", "")
raw_schedule_df['Away_Team'] = raw_schedule_df['Away_Team'].str.replace("'", "")

# Helper function to calculate distance between two sets of coordinates
def calculate_distance(coord1, coord2):
    if pd.notna(coord1[0]) and pd.notna(coord2[0]):
        return geodesic(coord1, coord2).miles
    return 0

# Step 1: Filter out exhibition games, games with "TBA", and games with "/" in team names
raw_schedule_filtered = raw_schedule_df[
    (raw_schedule_df['Conference'] != 'Exhibition') &
    (~raw_schedule_df['Away_Team'].str.contains('TBA')) &
    (~raw_schedule_df['Home_Team'].str.contains('TBA')) &
    (~raw_schedule_df['Away_Team'].str.contains('/')) &
    (~raw_schedule_df['Home_Team'].str.contains('/'))
]

# Step 2: Flag neutral site games using the 'Flag' column from the neutral arenas table
def is_neutral_game(row, flags):
    # Check for flags in both Conference and Game_Notes columns
    conference_match = any(flag in str(row['Conference']) for flag in flags)
    notes_match = any(flag in str(row['Game_Notes']) for flag in flags)
    return conference_match or notes_match

neutral_flags = neutral_arenas_df['Flag'].tolist()
raw_schedule_filtered['Is_Neutral_Game'] = raw_schedule_filtered.apply(is_neutral_game, axis=1, flags=neutral_flags)

# Step 3: Merge arena coordinates for home and away teams
# Merge home team coordinates
schedule_with_coords = raw_schedule_filtered.merge(arena_info_df[['Team', 'Latitude', 'Longitude']], 
                                                   left_on='Home_Team', right_on='Team', how='left')
schedule_with_coords = schedule_with_coords.rename(columns={'Latitude': 'Home_Lat', 'Longitude': 'Home_Lon'})

# Merge away team coordinates
schedule_with_coords = schedule_with_coords.merge(arena_info_df[['Team', 'Latitude', 'Longitude']], 
                                                  left_on='Away_Team', right_on='Team', how='left')
schedule_with_coords = schedule_with_coords.rename(columns={'Latitude': 'Away_Lat', 'Longitude': 'Away_Lon'})

# Step 4: Categorize games into on-campus conference, on-campus non-conference, and neutral site
def categorize_game(row):
    if row['Is_Neutral_Game']:
        return 'Neutral'
    elif row['Conference'] == 'Non-Conference':
        return 'On-Campus Non-Conference'
    else:
        return 'On-Campus Conference'

schedule_with_coords['Game_Type'] = schedule_with_coords.apply(categorize_game, axis=1)

# Step 5: Calculate the distance for all games
schedule_with_coords['Distance'] = schedule_with_coords.apply(
    lambda row: calculate_distance((row['Away_Lat'], row['Away_Lon']), (row['Home_Lat'], row['Home_Lon'])), axis=1)

# Adjust the neutral site distance calculation using the neutral arena coordinates for both teams
def calculate_neutral_distance(row, neutral_df):
    if row['Is_Neutral_Game']:
        # Find the neutral site coordinates from the neutral arenas table
        neutral_site = neutral_df[neutral_df['Flag'].apply(lambda x: x in str(row['Conference']) or x in str(row['Game_Notes']))]
        
        if not neutral_site.empty:
            neutral_lat = neutral_site.iloc[0]['latitude']
            neutral_lon = neutral_site.iloc[0]['longitude']
            # Calculate distance from away team to neutral site
            away_to_neutral = calculate_distance((row['Away_Lat'], row['Away_Lon']), (neutral_lat, neutral_lon))
            # Calculate distance from home team to neutral site
            home_to_neutral = calculate_distance((row['Home_Lat'], row['Home_Lon']), (neutral_lat, neutral_lon))
            # Return both distances separately to track for both teams
            return away_to_neutral, home_to_neutral
    # Non-neutral games: apply the same distance for both teams (travel for the away team)
    return row['Distance'], row['Distance']

# Apply the new logic for neutral site games using the correct neutral coordinates
schedule_with_coords[['Away_Distance', 'Home_Distance']] = schedule_with_coords.apply(calculate_neutral_distance, axis=1, result_type='expand', neutral_df=neutral_arenas_df)

# Convert 'Date' column to datetime format for easier manipulation
schedule_with_coords['Date'] = pd.to_datetime(schedule_with_coords['Date'])


# # Rework the Travel Flag logic to account for consecutive games at the same venue and not count them as travel
# schedule_with_coords['Travel_Flag'] = 1

# # Sort the data by 'Away_Team', 'Home_Team', and 'Date' to ensure games are grouped correctly
# schedule_with_coords = schedule_with_coords.sort_values(by=['Away_Team', 'Home_Team', 'Date'])



## Step 6: Calculate the travel flag for each game
# Reset the travel flag
schedule_with_coords['Travel_Flag'] = 1

# Sort the data by 'Away_Team', 'Home_Team', and 'Date' to ensure games are grouped correctly
schedule_with_coords = schedule_with_coords.sort_values(by=['Away_Team', 'Home_Team', 'Date'])

# Group by 'Away_Team' and 'Home_Team', then iterate through each group to set the travel flag
for (away_team, home_team), group in schedule_with_coords.groupby(['Away_Team', 'Home_Team']):
    group = group.sort_values(by='Date')  # Sort by date within each group
    
    # Iterate through the group to check for consecutive games
    for i in range(1, len(group)):
        current_game = group.iloc[i]
        previous_game = group.iloc[i - 1]
        
        # Check if the games are within (plus or minus) 3 days at the same venue
        if (current_game['Date'] - previous_game['Date']).days <= 3 and current_game['Distance'] == 0:
            # Set the travel flag to 0 for the current game
            schedule_with_coords.loc[current_game.name, 'Travel_Flag'] = 0
        

# # Only consider rows where Travel_Flag is 1 for calculating total travel distance
schedule_with_coords['Adjusted_Away_Distance'] = schedule_with_coords['Away_Distance'] * schedule_with_coords['Travel_Flag']
schedule_with_coords['Adjusted_Home_Distance'] = schedule_with_coords['Home_Distance'] * schedule_with_coords['Travel_Flag']





# Step 8: Group by team and game type for both away and home travel distances
away_team_travel_stats = schedule_with_coords.groupby(['Away_Team', 'Game_Type']).agg(
    total_trips=('Game_ID', 'count'),
    total_distance=('Adjusted_Away_Distance', 'sum'),
    average_distance=('Adjusted_Away_Distance', 'mean'),
    longest_trip=('Adjusted_Away_Distance', 'max'),
    shortest_trip=('Adjusted_Away_Distance', 'min')
).reset_index()

# Home teams need separate stats for neutral games
home_team_travel_stats = schedule_with_coords[schedule_with_coords['Is_Neutral_Game']].groupby(['Home_Team', 'Game_Type']).agg(
    total_trips=('Game_ID', 'count'),
    total_distance=('Adjusted_Home_Distance', 'sum'),
    average_distance=('Adjusted_Home_Distance', 'mean'),
    longest_trip=('Adjusted_Home_Distance', 'max'),
    shortest_trip=('Adjusted_Home_Distance', 'min')
).reset_index()

# Step 9: Combine both the away and home team travel stats into a unified DataFrame
travel_stats_combined = pd.concat([away_team_travel_stats.rename(columns={'Away_Team': 'Team'}),
                                   home_team_travel_stats.rename(columns={'Home_Team': 'Team'})], axis=0)

# Step 10: Aggregate the final travel stats for each team
team_travel_stats_final = travel_stats_combined.groupby(['Team', 'Game_Type']).agg(
    total_trips=('total_trips', 'sum'),
    total_distance=('total_distance', 'sum'),
    average_distance=('average_distance', 'mean'),
    longest_trip=('longest_trip', 'max'),
    shortest_trip=('shortest_trip', 'min')
).reset_index()

# Step 11: Pivot the final travel stats to match the required format
team_travel_summary_final = team_travel_stats_final.pivot_table(
    index='Team',
    columns='Game_Type',
    values=['total_trips', 'total_distance', 'average_distance', 'longest_trip', 'shortest_trip'],
    fill_value=0
)

# Step 12: Flatten the columns for readability
team_travel_summary_final.columns = ['_'.join(col).strip() for col in team_travel_summary_final.columns]

# Step 13: Rename and reorder columns to the desired format
team_travel_summary_reset = team_travel_summary_final.reset_index()

new_column_names_final = {
    'total_trips_Neutral': 'N_trips',
    'total_distance_Neutral': 'N_total_distance',
    'average_distance_Neutral': 'N_AVG',
    'longest_trip_Neutral': 'N_longest',
    'shortest_trip_Neutral': 'N_shortest',
    'total_trips_On-Campus Non-Conference': 'non_con_trips',
    'total_distance_On-Campus Non-Conference': 'non_con_total_distance',
    'average_distance_On-Campus Non-Conference': 'non_con_AVG',
    'longest_trip_On-Campus Non-Conference': 'non_con_longest',
    'shortest_trip_On-Campus Non-Conference': 'non_con_shortest',
    'total_trips_On-Campus Conference': 'con_trips',
    'total_distance_On-Campus Conference': 'con_total_distance',
    'average_distance_On-Campus Conference': 'con_AVG',
    'longest_trip_On-Campus Conference': 'con_longest',
    'shortest_trip_On-Campus Conference': 'con_shortest'
}

team_travel_summary_reset.rename(columns=new_column_names_final, inplace=True)


new_column_names_final = {
    'total_trips_Neutral': 'N_trips',
    'total_distance_Neutral': 'N_total_distance',
    'average_distance_Neutral': 'N_AVG',
    'longest_trip_Neutral': 'N_longest',
    'shortest_trip_Neutral': 'N_shortest',
    'total_trips_On-Campus Non-Conference': 'non_con_trips',
    'total_distance_On-Campus Non-Conference': 'non_con_total_distance',
    'average_distance_On-Campus Non-Conference': 'non_con_AVG',
    'longest_trip_On-Campus Non-Conference': 'non_con_longest',
    'shortest_trip_On-Campus Non-Conference': 'non_con_shortest',
    'total_trips_On-Campus Conference': 'con_trips',
    'total_distance_On-Campus Conference': 'con_total_distance',
    'average_distance_On-Campus Conference': 'con_AVG',
    'longest_trip_On-Campus Conference': 'con_longest',
    'shortest_trip_On-Campus Conference': 'con_shortest'
}

team_travel_summary_reset.rename(columns=new_column_names_final, inplace=True)

# Final columns order
columns_order_final = [
    'Team', 'N_trips', 'N_total_distance', 'N_AVG', 'N_longest', 'N_shortest',
    'non_con_trips', 'non_con_total_distance', 'non_con_AVG', 'non_con_longest', 'non_con_shortest',
    'con_trips', 'con_total_distance', 'con_AVG', 'con_longest', 'con_shortest'
]

team_travel_summary_final_display = team_travel_summary_reset[columns_order_final]

# Display the final table


team_travel_summary_final_display.head()




C:\Users\jbanc\AppData\Local\Temp\ipykernel_20260\3456361135.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_schedule_filtered['Is_Neutral_Game'] = raw_schedule_filtered.apply(is_neutral_game, axis=1, flags=neutral_flags)


,Team,N_trips,N_total_distance,N_AVG,N_longest,N_shortest,non_con_trips,non_con_total_distance,non_con_AVG,non_con_longest,non_con_shortest,con_trips,con_total_distance,con_AVG,con_longest,con_shortest
0,Air Force,1.0,600.521127,600.521127,600.521127,600.521127,3.0,3542.001054,1180.667018,1764.998085,12.004884,12.0,18965.356972,1580.446414,1771.362615,1316.604241
1,Alaska,1.0,2721.985269,2721.985269,2721.985269,2721.985269,23.0,57082.542783,2481.849686,3275.395086,260.422806,0.0,0.000000,0.000000,0.000000,0.000000
2,Alaska Anchorage,0.0,0.000000,0.000000,0.000000,0.000000,20.0,53129.593788,2656.479689,3401.368614,260.422806,0.0,0.000000,0.000000,0.000000,0.000000
3,American Intl,0.0,0.000000,0.000000,0.000000,0.000000,6.0,1611.208703,268.534784,565.237531,51.976219,13.0,2582.489109,198.653008,410.160970,39.218970
4,Arizona State,1.0,1.751265,1.751265,1.751265,1.751265,6.0,8786.443755,1464.407292,2271.357695,548.654698,12.0,10198.994030,849.916169,1563.331540,0.000000


In [47]:
# Show only Michginan teams to quality check
team_travel_summary_final_display[team_travel_summary_final_display['Team'].str.contains('Michigan')]

,Team,N_trips,N_total_distance,N_AVG,N_longest,N_shortest,non_con_trips,non_con_total_distance,non_con_AVG,non_con_longest,non_con_shortest,con_trips,con_total_distance,con_AVG,con_longest,con_shortest
34,Michigan,2.0,237.861534,118.930767,202.363751,35.497783,5.0,4681.364810,936.272962,1646.638250,95.807984,12.0,2939.925058,244.993755,510.848289,49.682849
35,Michigan State,3.0,310.955628,103.651876,170.821767,62.204115,4.0,1092.667842,273.166961,286.541055,259.792866,11.0,2845.336209,258.666928,463.104574,49.682849
36,Michigan Tech,2.0,2035.046257,1017.523128,1715.855635,319.190621,0.0,0.000000,0.000000,0.000000,0.000000,14.0,3822.215818,273.015416,469.624467,0.000000
44,Northern Michigan,1.0,262.673453,262.673453,262.673453,262.673453,3.0,2127.680969,709.226990,1030.803345,66.074279,14.0,4111.358643,293.668475,502.372833,66.074279
65,Western Michigan,1.0,46.901420,46.901420,46.901420,46.901420,3.0,931.591936,310.530645,738.973955,95.807984,12.0,8913.161448,742.763454,1553.692919,196.194384


In [48]:
### OUTPUT TO CSV
temp_path = os.path.join('..', 'TEMP', 'team_travel_summary_final_v1.01.csv')
team_travel_summary_final_display.to_csv(temp_path, index=False)


In [49]:
break


### OLDER CODE 

In [50]:


from geopy.distance import geodesic

# ## Remove ' from all Team columns
raw_schedule_df['Home_Team'] = raw_schedule_df['Home_Team'].str.replace("'", "")
raw_schedule_df['Away_Team'] = raw_schedule_df['Away_Team'].str.replace("'", "")


# Helper function to calculate distance between two sets of coordinates
def calculate_distance(coord1, coord2):
    if pd.notna(coord1[0]) and pd.notna(coord2[0]):
        return geodesic(coord1, coord2).miles
    return 0

# Step 1: Filter out exhibition games, games with "TBA", and games with "/" in team names
raw_schedule_filtered = raw_schedule_df[
    (raw_schedule_df['Conference'] != 'Exhibition') &
    (~raw_schedule_df['Away_Team'].str.contains('TBA')) &
    (~raw_schedule_df['Home_Team'].str.contains('TBA')) &
    (~raw_schedule_df['Away_Team'].str.contains('/')) &
    (~raw_schedule_df['Home_Team'].str.contains('/'))
]

# Step 2: Flag neutral site games using the 'Flag' column from the neutral arenas table
def is_neutral_game(row, flags):
    # Check for flags in both Conference and Game_Notes columns
    conference_match = any(flag in str(row['Conference']) for flag in flags)
    notes_match = any(flag in str(row['Game_Notes']) for flag in flags)
    return conference_match or notes_match

neutral_flags = neutral_arenas_df['Flag'].tolist()
raw_schedule_filtered['Is_Neutral_Game'] = raw_schedule_filtered.apply(is_neutral_game, axis=1, flags=neutral_flags)

# Step 3: Merge arena coordinates for home and away teams
# Merge home team coordinates
schedule_with_coords = raw_schedule_filtered.merge(arena_info_df[['Team', 'Latitude', 'Longitude']], 
                                                   left_on='Home_Team', right_on='Team', how='left')
schedule_with_coords = schedule_with_coords.rename(columns={'Latitude': 'Home_Lat', 'Longitude': 'Home_Lon'})

# Merge away team coordinates
schedule_with_coords = schedule_with_coords.merge(arena_info_df[['Team', 'Latitude', 'Longitude']], 
                                                  left_on='Away_Team', right_on='Team', how='left')
schedule_with_coords = schedule_with_coords.rename(columns={'Latitude': 'Away_Lat', 'Longitude': 'Away_Lon'})

# Step 4: Categorize games into on-campus conference, on-campus non-conference, and neutral site
def categorize_game(row):
    if row['Is_Neutral_Game']:
        return 'Neutral'
    elif row['Conference'] == 'Non-Conference':
        return 'On-Campus Non-Conference'
    else:
        return 'On-Campus Conference'

schedule_with_coords['Game_Type'] = schedule_with_coords.apply(categorize_game, axis=1)

# Adjust the neutral site distance calculation to record distances for both teams separately

def calculate_neutral_distance(row):
    if row['Is_Neutral_Game']:
        # Calculate distance from away team to neutral site (assuming the home location in the row represents the neutral site)
        away_to_neutral = calculate_distance((row['Away_Lat'], row['Away_Lon']), (row['Home_Lat'], row['Home_Lon']))
        
        # Calculate distance from home team to neutral site (assuming the home location in the row represents the neutral site)
        home_to_neutral = calculate_distance((row['Home_Lat'], row['Home_Lon']), (row['Away_Lat'], row['Away_Lon']))
        
        # Return both distances separately to track for both teams
        return away_to_neutral, home_to_neutral
    else:
        return row['Distance'], row['Distance']  # Non-neutral games, the same distance applies for both (travel for the away team)

# Apply the new logic for neutral site games
schedule_with_coords[['Away_Distance', 'Home_Distance']] = schedule_with_coords.apply(calculate_neutral_distance, axis=1, result_type='expand')

# Reset the travel flag
schedule_with_coords['Travel_Flag'] = 1

# Only consider trips where the Travel_Flag is set to 1 for calculating travel stats
schedule_with_coords['Adjusted_Away_Distance'] = schedule_with_coords['Away_Distance'] * schedule_with_coords['Travel_Flag']
schedule_with_coords['Adjusted_Home_Distance'] = schedule_with_coords['Home_Distance'] * schedule_with_coords['Travel_Flag']

# Group by team and game type for both away and home travel distances
away_team_travel_stats = schedule_with_coords.groupby(['Away_Team', 'Game_Type']).agg(
    total_trips=('Game_ID', 'count'),
    total_distance=('Adjusted_Away_Distance', 'sum'),
    average_distance=('Adjusted_Away_Distance', 'mean'),
    longest_trip=('Adjusted_Away_Distance', 'max'),
    shortest_trip=('Adjusted_Away_Distance', 'min')
).reset_index()

# Home teams need separate stats for neutral games
home_team_travel_stats = schedule_with_coords[schedule_with_coords['Is_Neutral_Game']].groupby(['Home_Team', 'Game_Type']).agg(
    total_trips=('Game_ID', 'count'),
    total_distance=('Adjusted_Home_Distance', 'sum'),
    average_distance=('Adjusted_Home_Distance', 'mean'),
    longest_trip=('Adjusted_Home_Distance', 'max'),
    shortest_trip=('Adjusted_Home_Distance', 'min')
).reset_index()

# Combine both the away and home team travel stats into a unified DataFrame
travel_stats_combined = pd.concat([away_team_travel_stats.rename(columns={'Away_Team': 'Team'}),
                                   home_team_travel_stats.rename(columns={'Home_Team': 'Team'})], axis=0)

# Aggregate the final travel stats for each team
team_travel_stats_final = travel_stats_combined.groupby(['Team', 'Game_Type']).agg(
    total_trips=('total_trips', 'sum'),
    total_distance=('total_distance', 'sum'),
    average_distance=('average_distance', 'mean'),
    longest_trip=('longest_trip', 'max'),
    shortest_trip=('shortest_trip', 'min')
).reset_index()

# Pivot the final travel stats to match the required format
team_travel_summary_final = team_travel_stats_final.pivot_table(
    index='Team',
    columns='Game_Type',
    values=['total_trips', 'total_distance', 'average_distance', 'longest_trip', 'shortest_trip'],
    fill_value=0
)

# Flatten the columns for readability
team_travel_summary_final.columns = ['_'.join(col).strip() for col in team_travel_summary_final.columns]

# Rename and reorder columns to the desired format
team_travel_summary_reset = team_travel_summary_final.reset_index()

new_column_names_final = {
    'total_trips_Neutral': 'N_trips',
    'total_distance_Neutral': 'N_total_distance',
    'average_distance_Neutral': 'N_AVG',
    'longest_trip_Neutral': 'N_longest',
    'shortest_trip_Neutral': 'N_shortest',
    'total_trips_On-Campus Non-Conference': 'non_con_trips',
    'total_distance_On-Campus Non-Conference': 'non_con_total_distance',
    'average_distance_On-Campus Non-Conference': 'non_con_AVG',
    'longest_trip_On-Campus Non-Conference': 'non_con_longest',
    'shortest_trip_On-Campus Non-Conference': 'non_con_shortest',
    'total_trips_On-Campus Conference': 'con_trips',
    'total_distance_On-Campus Conference': 'con_total_distance',
    'average_distance_On-Campus Conference': 'con_AVG',
    'longest_trip_On-Campus Conference': 'con_longest',
    'shortest_trip_On-Campus Conference': 'con_shortest'
}

team_travel_summary_reset.rename(columns=new_column_names_final, inplace=True)

# Final columns order
columns_order_final = [
    'Team', 'N_trips', 'N_total_distance', 'N_AVG', 'N_longest', 'N_shortest',
    'non_con_trips', 'non_con_total_distance', 'non_con_AVG', 'non_con_longest', 'non_con_shortest',
    'con_trips', 'con_total_distance', 'con_AVG', 'con_longest', 'con_shortest'
]

team_travel_summary_final_display = team_travel_summary_reset[columns_order_final]

# Display the final table


team_travel_summary_final_display.head()



C:\Users\jbanc\AppData\Local\Temp\ipykernel_20260\1617166954.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_schedule_filtered['Is_Neutral_Game'] = raw_schedule_filtered.apply(is_neutral_game, axis=1, flags=neutral_flags)


KeyError: 'Distance'

In [ ]:
# ## Remove ' from all Team columns
# raw_schedule_df['Home_Team'] = raw_schedule_df['Home_Team'].str.replace("'", "")
# raw_schedule_df['Away_Team'] = raw_schedule_df['Away_Team'].str.replace("'", "")


# # Step 1: Filter out exhibition games
# raw_schedule_filtered = raw_schedule_df[raw_schedule_df['Conference'] != 'Exhibition']

# # Step 2: Flag neutral site games based on the 'Flag' from the neutral arenas table
# def is_neutral_game(row, flags):
#     conference_match = any(flag in str(row['Conference']) for flag in flags)
#     notes_match = any(flag in str(row['Game_Notes']) for flag in flags)
#     return conference_match or notes_match

# # Extract the list of flags from the neutral arenas table
# neutral_flags = neutral_arenas_df['Flag'].tolist()

# # Apply the neutral site game flagging logic
# raw_schedule_filtered['Is_Neutral_Game'] = raw_schedule_filtered.apply(is_neutral_game, axis=1, flags=neutral_flags)

# # Step 3: Merge coordinates for home and away teams based on the arena info
# # First, merge the home team coordinates
# schedule_with_coords = raw_schedule_filtered.merge(arena_info_df[['Team', 'Latitude', 'Longitude']], 
#                                                    left_on='Home_Team', right_on='Team', how='left')
# schedule_with_coords = schedule_with_coords.rename(columns={'Latitude': 'Home_Lat', 'Longitude': 'Home_Lon'})

# # Then, merge the away team coordinates
# schedule_with_coords = schedule_with_coords.merge(arena_info_df[['Team', 'Latitude', 'Longitude']], 
#                                                   left_on='Away_Team', right_on='Team', how='left')
# schedule_with_coords = schedule_with_coords.rename(columns={'Latitude': 'Away_Lat', 'Longitude': 'Away_Lon'})

# # Step 4: Categorize games into three categories: on-campus conference, on-campus non-conference, and neutral site
# def categorize_game(row):
#     if row['Is_Neutral_Game']:
#         return 'Neutral'
#     elif row['Conference'] == 'Non-Conference':
#         return 'On-Campus Non-Conference'
#     else:
#         return 'On-Campus Conference'

# schedule_with_coords['Game_Type'] = schedule_with_coords.apply(categorize_game, axis=1)

# # View the updated table with neutral game flagging and game categorization
# schedule_with_coords[['Date', 'Away_Team', 'Home_Team', 'Is_Neutral_Game', 'Game_Type']].head()


In [384]:
# from geopy.distance import geodesic

# # Helper function to calculate distance between two sets of coordinates
# def calculate_distance(coord1, coord2):
#     if pd.notna(coord1[0]) and pd.notna(coord2[0]):  # Ensure coordinates are valid
#         return geodesic(coord1, coord2).miles
#     return 0

# # Step 5: Calculate distance for each game
# schedule_with_coords['Distance'] = schedule_with_coords.apply(
#     lambda row: calculate_distance((row['Away_Lat'], row['Away_Lon']), (row['Home_Lat'], row['Home_Lon'])), axis=1)

# # Step 6: Group by team and game type to calculate total trips, total distance, average distance, longest trip, shortest trip
# team_travel_stats = schedule_with_coords.groupby(['Away_Team', 'Game_Type']).agg(
#     total_trips=('Game_ID', 'count'),
#     total_distance=('Distance', 'sum'),
#     average_distance=('Distance', 'mean'),
#     longest_trip=('Distance', 'max'),
#     shortest_trip=('Distance', 'min')
# ).reset_index()

# # Pivot the table to show the results for each game type
# team_travel_summary = team_travel_stats.pivot_table(
#     index='Away_Team', 
#     columns='Game_Type', 
#     values=['total_trips', 'total_distance', 'average_distance', 'longest_trip', 'shortest_trip'], 
#     fill_value=0
# )

# # Flatten the multi-index columns for readability
# team_travel_summary.columns = ['_'.join(col).strip() for col in team_travel_summary.columns]

# # team_travel_summary.head(25)


In [ ]:
# Step 1: Convert 'Date' column to datetime format for easier manipulation
schedule_with_coords['Date'] = pd.to_datetime(schedule_with_coords['Date'])

# Step 2: Initialize the Travel_Flag for every game as 1 (indicating travel)
schedule_with_coords['Travel_Flag'] = 1

# Step 3: Sort by 'Away_Team', 'Home_Team', and 'Date' to group correctly
schedule_with_coords = schedule_with_coords.sort_values(by=['Away_Team', 'Home_Team', 'Date'])

# Step 4: Group by 'Away_Team' and 'Home_Team', and identify multiple-game series
for (away_team, home_team), group in schedule_with_coords.groupby(['Away_Team', 'Home_Team']):
    group = group.sort_values(by='Date')  # Sort by date within each group
    
    # Iterate through the group to check for consecutive games within 3 days
    for i in range(1, len(group)):
        current_game = group.iloc[i]
        previous_game = group.iloc[i - 1]
        
        # If games are within 3 days, set the travel flag of the current game to 0 (no new trip)
        if (current_game['Date'] - previous_game['Date']).days <= 3:
            schedule_with_coords.loc[current_game.name, 'Travel_Flag'] = 0

# Step 5: Calculate adjusted travel distance, considering only trips with Travel_Flag set to 1
schedule_with_coords['Adjusted_Travel_Distance'] = schedule_with_coords['Distance'] * schedule_with_coords['Travel_Flag']

# Step 6: Remove games with '/' in the team names (as per original notebook logic)
schedule_with_coords = schedule_with_coords[~schedule_with_coords['Home_Team'].str.contains('/')]
schedule_with_coords = schedule_with_coords[~schedule_with_coords['Away_Team'].str.contains('/')]

# Step 7: Handle neutral site games by calculating distance for both home and away teams
def calculate_neutral_distance(row):
    if row['Is_Neutral_Game']:
        # Calculate distance from both home and away team to the neutral site
        away_to_neutral = calculate_distance((row['Away_Lat'], row['Away_Lon']), (row['Home_Lat'], row['Home_Lon']))
        home_to_neutral = calculate_distance((row['Home_Lat'], row['Home_Lon']), (row['Away_Lat'], row['Away_Lon']))
        return away_to_neutral + home_to_neutral  # Sum of both distances
    return row['Distance']  # Use original distance if not a neutral site game

schedule_with_coords['Adjusted_Travel_Distance'] = schedule_with_coords.apply(calculate_neutral_distance, axis=1)

# Step 8: Group by team and game type, considering only games with Travel_Flag set to 1
team_travel_stats_updated = schedule_with_coords[schedule_with_coords['Travel_Flag'] == 1].groupby(['Away_Team', 'Game_Type']).agg(
    total_trips=('Game_ID', 'count'),
    total_distance=('Adjusted_Travel_Distance', 'sum'),
    average_distance=('Adjusted_Travel_Distance', 'mean'),
    longest_trip=('Adjusted_Travel_Distance', 'max'),
    shortest_trip=('Adjusted_Travel_Distance', 'min')
).reset_index()

# Step 9: Pivot the table to show the results for each game type
team_travel_summary_updated = team_travel_stats_updated.pivot_table(
    index='Away_Team', 
    columns='Game_Type', 
    values=['total_trips', 'total_distance', 'average_distance', 'longest_trip', 'shortest_trip'], 
    fill_value=0
)

# Flatten the multi-index columns for readability
team_travel_summary_updated.columns = ['_'.join(col).strip() for col in team_travel_summary_updated.columns]

# Rename and rearrange columns as requested
team_travel_summary_reset_updated = team_travel_summary_updated.reset_index()

new_column_names_updated = {
    'total_trips_Neutral': 'N_trips',
    'total_distance_Neutral': 'N_total_distance',
    'average_distance_Neutral': 'N_AVG',
    'longest_trip_Neutral': 'N_longest',
    'shortest_trip_Neutral': 'N_shortest',
    'total_trips_On-Campus Non-Conference': 'non_con_trips',
    'total_distance_On-Campus Non-Conference': 'non_con_total_distance',
    'average_distance_On-Campus Non-Conference': 'non_con_AVG',
    'longest_trip_On-Campus Non-Conference': 'non_con_longest',
    'shortest_trip_On-Campus Non-Conference': 'non_con_shortest',
    'total_trips_On-Campus Conference': 'con_trips',
    'total_distance_On-Campus Conference': 'con_total_distance',
    'average_distance_On-Campus Conference': 'con_AVG',
    'longest_trip_On-Campus Conference': 'con_longest',
    'shortest_trip_On-Campus Conference': 'con_shortest'
}

team_travel_summary_reset_updated.rename(columns=new_column_names_updated, inplace=True)

columns_order_updated = [
    'Away_Team', 'N_trips', 'N_total_distance', 'N_AVG', 'N_longest', 'N_shortest',
    'non_con_trips', 'non_con_total_distance', 'non_con_AVG', 'non_con_longest', 'non_con_shortest',
    'con_trips', 'con_total_distance', 'con_AVG', 'con_longest', 'con_shortest'
]

team_travel_summary_final_updated = team_travel_summary_reset_updated[columns_order_updated]

# Display the final updated table


team_travel_summary_final_updated.tail(45)



In [ ]:
# # Let's rename and rearrange the columns to match the desired output format
# team_travel_summary_reset = team_travel_summary.reset_index()

# # New column names for better readability
# new_column_names = {
#     'total_trips_Neutral': 'N_trips',
#     'total_distance_Neutral': 'N_total_distance',
#     'average_distance_Neutral': 'N_AVG',
#     'longest_trip_Neutral': 'N_longest',
#     'shortest_trip_Neutral': 'N_shortest',
#     'total_trips_On-Campus Non-Conference': 'non_con_trips',
#     'total_distance_On-Campus Non-Conference': 'non_con_total_distance',
#     'average_distance_On-Campus Non-Conference': 'non_con_AVG',
#     'longest_trip_On-Campus Non-Conference': 'non_con_longest',
#     'shortest_trip_On-Campus Non-Conference': 'non_con_shortest',
#     'total_trips_On-Campus Conference': 'con_trips',
#     'total_distance_On-Campus Conference': 'con_total_distance',
#     'average_distance_On-Campus Conference': 'con_AVG',
#     'longest_trip_On-Campus Conference': 'con_longest',
#     'shortest_trip_On-Campus Conference': 'con_shortest'
# }

# # Renaming the columns
# team_travel_summary_reset.rename(columns=new_column_names, inplace=True)

# # Rearranging columns
# columns_order = [
#     'Away_Team', 'N_trips', 'N_total_distance', 'N_AVG', 'N_longest', 'N_shortest',
#     'non_con_trips', 'non_con_total_distance', 'non_con_AVG', 'non_con_longest', 'non_con_shortest',
#     'con_trips', 'con_total_distance', 'con_AVG', 'con_longest', 'con_shortest'
# ]

# team_travel_summary_final = team_travel_summary_reset[columns_order]

# # Display the final table
# # tools.display_dataframe_to_user(name="Team Travel Summary (Renamed and Rearranged)", dataframe=team_travel_summary_final)

# team_travel_summary_final.head()


## ORIGINAL CODE

### Initial Transformation


In [363]:
### HOTFIX
### remove ' from Team names
schedule_data['Home_Team'] = schedule_data['Home_Team'].str.replace("'", "")
schedule_data['Away_Team'] = schedule_data['Away_Team'].str.replace("'", "")

##### Drop Exhibition Games

In [ ]:
## Drop Exhibition games from schedule
# If 'Exhibition' in Conference column, drop row

# Print Schedule length
print(f"Schedule length before dropping Exhibition games: {len(schedule_data)}")

# Drop rows with 'Exhibition' in Conference column
schedule_data = schedule_data[schedule_data['Conference'] != 'Exhibition']

# Print Schedule length
print(f"Schedule length after dropping Exhibition games: {len(schedule_data)}")


## Account for neutral site games

In [365]:
# Identify rows in the schedule table that involve neutral site games
# by checking if the 'Conference' or 'Game_Notes' contains a match with the 'Flag' in the neutral arenas table.

# Helper function to find if any flag appears in the Conference or Game_Notes columns
def is_neutral_game(row, flags):
    # Check for flags in both Conference and Game_Notes, ensuring correct handling of NaN
    conference_match = any(flag in str(row['Conference']) for flag in flags)
    notes_match = any(flag in str(row['Game_Notes']) for flag in flags)
    return conference_match or notes_match

# Extract the list of flags from the neutral arenas table (assuming already loaded neutral_arenas_df)
neutral_flags = neutral_arenas_df['Flag'].tolist()

# Apply the function to the schedule data to identify neutral site games
schedule_data['Is_Neutral_Game'] = schedule_data.apply(is_neutral_game, axis=1, flags=neutral_flags)

# Filter the schedule for neutral site games
neutral_site_games = schedule_data[schedule_data['Is_Neutral_Game']]

### Calculate the distance to neutral site locations

In [366]:
### refactor the code to include the distance between the two teams


# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to merge team location data and neutral arenas
def merge_team_and_arena_data(schedule_df, team_df, arena_df):
    # Merge team locations (away and home teams)
    schedule_df = schedule_df.merge(
        team_df[['Team', 'Latitude', 'Longitude']], 
        left_on='Away_Team', right_on='Team', how='left', suffixes=('', '_away')
    )
    schedule_df = schedule_df.merge(
        team_df[['Team', 'Latitude', 'Longitude']], 
        left_on='Home_Team', right_on='Team', how='left', suffixes=('', '_home')
    )

    # Merge on 'Conference' column first
    schedule_df = schedule_df.merge(
        arena_df[['Flag', 'latitude', 'longitude']], 
        left_on='Conference', right_on='Flag', how='left'
    )

    # Handle rows where the Conference merge did not work by checking 'Game_Notes'
    missing_coords_df = schedule_df[schedule_df['latitude'].isnull()].copy()

    def match_flag_in_game_notes(row, arena_df):
        for _, flag_row in arena_df.iterrows():
            if flag_row['Flag'] in str(row['Game_Notes']):
                return flag_row['latitude'], flag_row['longitude']
        return None, None

    # Apply partial matching function for missing coordinates
    missing_coords_df[['latitude', 'longitude']] = missing_coords_df.apply(
        lambda row: match_flag_in_game_notes(row, arena_df), axis=1, result_type="expand"
    )

    # Fill missing latitude/longitude
    schedule_df.loc[schedule_df['latitude'].isnull(), ['latitude', 'longitude']] = missing_coords_df[['latitude', 'longitude']]

    return schedule_df

# Output results
# output_path = '../TEMP/neutral_site_games_distances_TEST2.csv'
# neutral_site_games.to_csv(output_path, index=False)

## PRINT DF INFOR BEFORE THE CLEANING
# print(neutral_site_games.info())

In [367]:
### CLEANING BASED ON LOOK AT OUTPUT FROM CELL ABOVE


# Remove Rows that have TBD or a / in one of the team columns
neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('/')]
neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('/')]
neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('TBD')]
neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('TBD')]

# Print DF INFO AFTER CLEANING
# print(neutral_site_games.info())

# OUTPUT CSV FOR CHECKING INTO TEMP FOLDER
# output_path = '../TEMP/neutral_site_games.csv'
# neutral_site_games.to_csv(output_path, index=False)

In [368]:
# neutral_site_games.head()

In [369]:
## New Approach - create a new table and structure for the neutral site games - add them to the agg count at the end

## Assign a location to each game - based on the FLag column from neutral_arenas_df

# Drop the columns that are not needed
neutral_site_games = neutral_site_games.drop(columns=['Away_Team_Link', 'Away_Score', 'Home_Team_Link', 'Home_Score', 'OT', 'Box_Link', 'Metrics_Link'])

# Reinex the DF
neutral_site_games.reset_index(drop=True, inplace=True)

# If Game_Notes is NaN fill with Conference
neutral_site_games['Game_Notes'] = neutral_site_games['Game_Notes'].fillna(neutral_site_games['Conference'])

# Assign a location to each game
# Look for Game_Notes that contain the Flag from neutral_arenas_df
# If there is a match, assign the location to the game

# Helper function to assign latitude and longitude to each game
def assign_location(row, arena_df):
    for _, flag_row in arena_df.iterrows():
        if flag_row['Flag'] in str(row['Game_Notes']):
            return flag_row['latitude'], flag_row['longitude']
    return None, None

# Extract the list of flags from the neutral arenas table (assuming already loaded neutral_arenas_df)
neutral_flags = neutral_arenas_df['Flag'].tolist()

# Apply the function to the schedule data to identify neutral site games
neutral_site_games[['latitude', 'longitude']] = neutral_site_games.apply(assign_location, axis=1, arena_df=neutral_arenas_df, result_type="expand")

# If Gama_Notes is empty or an empty string, fill with Conference value
# neutral_site_games['Game_Notes'] = neutral_site_games['Game_Notes'].replace('', neutral_site_games['Conference']) # NOT WORKING AT ALL - THROWING ERROR
# neutral_site_games['Game_Notes'] = neutral_site_games['Game_Notes'].fillna(neutral_site_games['Conference']) # Not working must be empty string not NaN


# neutral_site_games.head()
# neutral_site_games.tail()
# neutral_site_games

In [ ]:
### Calculate the distance between the two teams for each game

# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to calculate distances to neutral site
def calculate_team_distances(schedule_df):
    schedule_df['Away_Distance'] = schedule_df.apply(
        lambda row: calculate_distance(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']), axis=1
    )
    schedule_df['Home_Distance'] = schedule_df.apply(
        lambda row: calculate_distance(row['Latitude_home'], row['Longitude_home'], row['latitude'], row['longitude']), axis=1
    )

    return schedule_df

# Refactor into steps
neutral_site_games = merge_team_and_arena_data(schedule_data, arena_data, neutral_arenas_df)
neutral_site_games = calculate_team_distances(neutral_site_games)

# Filter out rows with missing distances
neutral_site_games = neutral_site_games.dropna(subset=['Away_Distance', 'Home_Distance'])


# Output results
# output_path = '../TEMP/neutral_site_games_distances_TESTv3.csv'
# neutral_site_games.to_csv(output_path, index=False)

# neutral_site_games.head()
# neutral_site_games.tail()
# neutral_site_games.info()
# Show Michigan Tech games
neutral_site_games[neutral_site_games['Away_Team'] == 'Michigan Tech']

In [ ]:
### DESIRED OUTPUT
# Date, Game_ID, Conference, Game_Notes, Team, Distance for each team in each game

# Create a new DataFrame to store the results
neutral_site_games_agg = pd.DataFrame(columns=['Date', 'Game_ID', 'Game_Notes', 'Team', 'N_Distance'])

# Iterate through the neutral_site_games DataFrame and add the data to the new DataFrame
rows = []  # Use a list to accumulate rows for better performance
for index, row in neutral_site_games.iterrows():
    # Add the Away Team data
    rows.append({
        'Date': row['Date'],
        'Game_ID': row['Game_ID'],
        'Game_Notes': row['Game_Notes'],
        'Team': row['Away_Team'],
        'N_Distance': row['Away_Distance']
    })
    
    # Add the Home Team data
    rows.append({
        'Date': row['Date'],
        'Game_ID': row['Game_ID'],
        'Game_Notes': row['Game_Notes'],
        'Team': row['Home_Team'],
        'N_Distance': row['Home_Distance']
    })

# Convert the list of rows into a DataFrame and concatenate
neutral_site_games_agg_1 = pd.concat([neutral_site_games_agg, pd.DataFrame(rows)], ignore_index=True)

# Create a new DataFrame to store the results
## Team, Nuetral Site Games, Longest Nueteral Site Game, Shortest Neutral Site Game, Total Distance, Average Neutral Site Game Distance
neutral_site_games_agg_1 = neutral_site_games_agg_1.groupby('Team').agg(
    N_Games=('Game_ID', 'nunique'),
    Longest_N_Game=('N_Distance', 'max'),
    Shortest_N_Game=('N_Distance', 'min'),
    Total_N_Distance=('N_Distance', 'sum'),
    Average_N_Distance=('N_Distance', 'mean')
).reset_index()


# Output results
output_path = '../TEMP/neutral_site_games_agg.csv'
neutral_site_games_agg_1.to_csv(output_path, index=False)

# Display the first and last few rows of the DataFrame
# neutral_site_games_agg_1.head(23)
neutral_site_games_agg_1.tail()
# Show Michigan Tech
neutral_site_games_agg_1[neutral_site_games_agg_1['Team'] == 'Michigan Tech']

## NON NEUTRAL GAMES

In [372]:
# Merge the schedule data with the arena data to include home and away team locations

# First, ensure team names match between datasets
# We will merge on the 'Team' column in the arena data and 'Home_Team'/'Away_Team' in the schedule data
merged_data = schedule_data.merge(arena_data[['Team', 'Latitude', 'Longitude']], left_on='Home_Team', right_on='Team', how='left')
merged_data = merged_data.rename(columns={'Latitude': 'Home_Latitude', 'Longitude': 'Home_Longitude'})

# Merge again for the away teams
merged_data = merged_data.merge(arena_data[['Team', 'Latitude', 'Longitude']], left_on='Away_Team', right_on='Team', how='left')
merged_data = merged_data.rename(columns={'Latitude': 'Away_Latitude', 'Longitude': 'Away_Longitude'})

# Drop the unnecessary 'Team' columns from the merged data
merged_data = merged_data.drop(columns=['Team_x', 'Team_y'])

# Display the first few rows of the merged data to verify the result
# merged_data.head()

### Calculate the distance between each school
- Using Haversine equation to calculate the straight line distance between two sets of lat/lon coodinates

In [373]:
## VERSION 2 USING GEOPY
# Function to calculate distance with NaN check
def calculate_distance(row):
    # Check for NaN values in lat/long coordinates
    if (np.isnan(row['Home_Latitude']) or np.isnan(row['Home_Longitude']) or
        np.isnan(row['Away_Latitude']) or np.isnan(row['Away_Longitude'])):
        return np.nan  # Return NaN if any of the coordinates are missing
    else:
        # Calculate the distance if all coordinates are present
        return geodesic((row['Home_Latitude'], row['Home_Longitude']), 
                        (row['Away_Latitude'], row['Away_Longitude'])).miles

# Apply the function to calculate the distance between the home and away arenas
merged_data['Distance_Miles'] = merged_data.apply(calculate_distance, axis=1)

# Display the updated data with the calculated distance
# merged_data[['Conference', 'Home_Team', 'Away_Team', 'Distance_Miles']].head()
## Display all columns of the merged data
# merged_data.head()


### Filter out results to avoid double counting games on weekend series

- Travel_Flag to account for consecutive games played at the same venue within a 3-day span. If a team plays multiple games at the same venue within this period, travel is only counted for the first game.

- The Adjusted_Travel_Distance column reflects the distance a team will travel for each game, considering the consecutive game rule.

In [ ]:
### VERSION 2
# Update logic to handle non-consecutive rows by grouping first

# Convert 'Date' column to datetime format for easier manipulation
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

# Reset the travel flag
merged_data['Travel_Flag'] = 1

# Sort the data by 'Away_Team', 'Home_Team', and 'Date' to ensure games are grouped correctly
merged_data = merged_data.sort_values(by=['Away_Team', 'Home_Team', 'Date'])

# Group by 'Away_Team' and 'Home_Team', then iterate through each group to set the travel flag
for (away_team, home_team), group in merged_data.groupby(['Away_Team', 'Home_Team']):
    group = group.sort_values(by='Date')  # Sort by date within each group
    
    # Iterate through the group to check for consecutive games
    for i in range(1, len(group)):
        current_game = group.iloc[i]
        previous_game = group.iloc[i - 1]
        
        # Check if the games are within 3 days
        if (current_game['Date'] - previous_game['Date']).days <= 3:
            # Set the travel flag to 0 for the current game
            merged_data.loc[current_game.name, 'Travel_Flag'] = 0

# Only consider rows where travel flag is 1 for calculating total travel distance
merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# Remove any row where there is a / in either the Home_Team or Away_Team
merged_data = merged_data[~merged_data['Home_Team'].str.contains('/')]
merged_data = merged_data[~merged_data['Away_Team'].str.contains('/')]

# Display the updated data with the travel flag and adjusted distance
merged_data[['Away_Team', 'Home_Team', 'Date', 'Distance_Miles', 'Travel_Flag', 'Adjusted_Travel_Distance']].head()

# OUTPUT TABLE TO TEMP FILE FOR TESTING
output_path = os.path.join('..', 'TEMP', 'schedule_w_distance.csv')
merged_data.to_csv(output_path, index=False)

schedule_w_distance = merged_data.copy()

## Aggregate Total Travel Distance for Each Team For Conference and Non Conference Games
## Calculate and store the Trip Count and the Average trip distance

In [305]:
# merged_data.head(15)\
# neutral_site_games.info()
# neutral_site_games.head(5)


In [ ]:
### Desired Output

## Team, Non-Conference_GP, Non-Conference_Miles, Non-Conference_AVG, 
## Conference_GP, Conference_Miles, Conference_AVG

# Only consider rows where travel flag is 1 for calculating total travel distance
merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# Filter Nuetral Site games out of the data because those distatnces are already calculated and will be merged back later
# If Game_ID appears in neutral_site_games, drop the row from merged_data
merged_data = merged_data[~merged_data['Game_ID'].isin(neutral_site_games['Game_ID'])]


# Group Non-Conference and Conference games separately
non_conference_data = merged_data[merged_data['Conference'] == 'Non-Conference']
conference_data = merged_data[merged_data['Conference'] != 'Non-Conference']



# Calculate total distance and number of games for each team in Non-Conference and Conference games
non_conference_agg = non_conference_data.groupby('Away_Team').agg(
    Non_Conference_GP=('Travel_Flag', 'sum'),
    Non_Conference_Miles=('Adjusted_Travel_Distance', 'sum'),
    Longest_NC_Game=('Adjusted_Travel_Distance', 'max'),
    Shortest_NC_Game=('Adjusted_Travel_Distance', lambda x: x[x > 0].min()) ## Find the Shortest Non 0 trip
    
).reset_index()

conference_agg = conference_data.groupby('Away_Team').agg(
    Conference_GP=('Travel_Flag', 'sum'),
    Conference_Miles=('Adjusted_Travel_Distance', 'sum'),
    Longest_Conf_Game=('Adjusted_Travel_Distance', 'max'),
    
    Shortest_Conf_Game=('Adjusted_Travel_Distance', lambda x: x[x > 0].min())
).reset_index()

# Calculate average distance per game for Non-Conference and Conference games
non_conference_agg['Non_Conference_AVG'] = non_conference_agg['Non_Conference_Miles'] / non_conference_agg['Non_Conference_GP']
conference_agg['Conference_AVG'] = conference_agg['Conference_Miles'] / conference_agg['Conference_GP']

### Merge the two tables to get the final output
merged_agg = non_conference_agg.merge(conference_agg, on='Away_Team', how='outer')

# Rename Away_Team to Team
merged_agg = merged_agg.rename(columns={'Away_Team': 'Team'})


# non_conference_agg.head(10) # Display the aggregated data for Non-Conference games
conference_agg.head(10) # Display the aggregated data for Conference games
merged_agg.head(10) # Display the merged aggregated data

## Examine Michigan State's Conference data
# Rearanging with travel flag up front
# conference_data = conference_data[['Away_Team', 'Home_Team', 'Date', 'Distance_Miles', 'Travel_Flag', 'Adjusted_Travel_Distance']]
# conference_data[conference_data['Away_Team'] == 'Michigan State']

non_conference_data.head()

# Find Michigan Tech in the nuetral site aggragated data
neutral_site_games_agg_1[neutral_site_games_agg_1['Team'] == 'Michigan Tech']
# Find Michigan Tech in nueutral site games
neutral_site_games[neutral_site_games['Away_Team'] == 'Michigan Tech']


In [307]:
## Merge the neutral site game table with the merged_agg table

# Merge the two tables to get the final output
final_agg = merged_agg.merge(neutral_site_games_agg_1, on='Team', how='outer')

In [308]:
neutral_site_games_agg_1
final_agg

# Output results for manual verification
output_path = '../TEMP/final_agg_v3.csv'
final_agg.to_csv(output_path, index=False)

In [ ]:
## Merge Nuetral Site Calculations with the rest of the data

# Merge the neutral site games data with the rest of the schedule data
merged_agg = merged_agg.merge(neutral_site_games_agg_1, left_on='Away_Team', right_on='Team', how='outer')


In [ ]:
########## OLD CODE
###### DOES NOT DIFFERENTIATE BETWEEN CONFERENCE AND NON-CONFERENCE GAMES
# # Only consider rows where travel flag is 1 for calculating total travel distance
# merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# # Calculate the total travel distance per team
# team_travel_distances = merged_data.groupby('Away_Team')['Adjusted_Travel_Distance'].sum().reset_index()
# team_travel_distances.columns = ['Team', 'Total_Travel_Distance']

# # Step 4: Adding Trip Count and Average Trip Distance

# # Calculate the number of trips for each team
# trip_count = merged_data[merged_data['Travel_Flag'] == 1].groupby('Away_Team').size().reset_index(name='Trip_Count')

# # Merge trip count with travel distances
# team_travel_data = pd.merge(team_travel_distances, trip_count, left_on='Team', right_on='Away_Team', how='left').drop(columns='Away_Team')

# # Calculate average trip distance
# team_travel_data['Average_Trip_Distance'] = team_travel_data['Total_Travel_Distance'] / team_travel_data['Trip_Count']

# # Display the top 5 teams with the highest average trip distance
# team_travel_data = team_travel_data.sort_values(by='Average_Trip_Distance', ascending=False)
# team_travel_data.head()

## Find The Closest Other Team to Each team and Store

In [76]:
from geopy.distance import geodesic
import pandas as pd

# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to find the closest team to a specific team in the arena data
def find_closest_team(current_team_row, team_data):
    # Initialize variables to store the closest team and distance
    closest_team = None
    closest_distance = np.inf
    
    # Iterate over each row in the team data
    for _, row in team_data.iterrows():
        # Calculate the distance between the current team and the other team
        distance = calculate_distance(current_team_row['Latitude'], current_team_row['Longitude'], row['Latitude'], row['Longitude'])
        
        # Update the closest team if the distance is smaller
        if distance is not None and distance < closest_distance:
            closest_team = row['Team']
            closest_distance = distance
    
    return closest_team, closest_distance

# Find the closest team to each team in the arena data
closest_teams = []
closest_distances = []

# Iterate over each row in the arena data to find the closest team
for _, row in arena_data.iterrows():
    # Exclude the current team from the comparison
    other_teams = arena_data[arena_data['Team'] != row['Team']]
    closest_team, closest_distance = find_closest_team(row, other_teams)
    
    closest_teams.append(closest_team)
    closest_distances.append(closest_distance)

# Add the closest team and distance to the arena data
arena_data['Closest_Team'] = closest_teams
arena_data['Closest_Distance'] = closest_distances

# Merge the closest team data with the team travel data
team_travel_data = team_travel_data.merge(arena_data[['Team', 'Closest_Team', 'Closest_Distance']], on='Team', how='left')

# Sort by closest distance to another team
team_travel_data = team_travel_data.sort_values(by='Closest_Distance')

# Display the top 5 teams with the highest average trip distance and closest team information
# team_travel_data.tail(25)

# Add The neutral site data to the aggrigated results

In [77]:
## REFACTOR

# Group by team to calculate total neutral site distances and count neutral site games

neutral_site_games_agg = neutral_site_games_agg_1.groupby('Team').agg({'N_Distance': 'sum', 'Game_ID': 'nunique'}).reset_index()
neutral_site_games_agg.rename(columns={'Game_ID': 'Neutral_Site_Trips'}, inplace=True)

# Merge the neutral site data (distances and game counts) with the team travel data
team_travel_data_refactored = pd.merge(team_travel_data, neutral_site_games_agg, on='Team', how='left')

# Fill missing values for teams without neutral site games
team_travel_data_refactored['N_Distance'] = team_travel_data_refactored['N_Distance'].fillna(0)
team_travel_data_refactored['Neutral_Site_Trips'] = team_travel_data_refactored['Neutral_Site_Trips'].fillna(0)

# Calculate regular trip stats (excluding neutral site games)
team_travel_data_refactored['Reg_Distance'] = team_travel_data_refactored['Total_Travel_Distance']
team_travel_data_refactored['Reg_Trips'] = team_travel_data_refactored['Trip_Count']
team_travel_data_refactored['Reg_AVG'] = team_travel_data_refactored['Reg_Distance'] / team_travel_data_refactored['Reg_Trips']

# Calculate total distance and average with neutral site trips included
team_travel_data_refactored['Total_Distance'] = team_travel_data_refactored['Reg_Distance'] + team_travel_data_refactored['N_Distance']
team_travel_data_refactored['N_AVG'] = team_travel_data_refactored['N_Distance'] / team_travel_data_refactored['Neutral_Site_Trips']
team_travel_data_refactored['Overall_AVG'] = team_travel_data_refactored['Total_Distance'] / (team_travel_data_refactored['Reg_Trips'] + team_travel_data_refactored['Neutral_Site_Trips'])

# Select and reorder the columns
team_travel_data_refactored = team_travel_data_refactored[[
    'Team', 
    'Reg_Distance', 
    'Reg_Trips', 
    'Reg_AVG', 
    'N_Distance', 
    'Neutral_Site_Trips', 
    'N_AVG', 
    'Total_Distance', 
    'Overall_AVG'
]]

# Add the Closest Team and CTeam_Distance columns back at the end of the table
team_travel_data_refactored = pd.merge(team_travel_data_refactored, 
                                       team_travel_data[['Team', 'Closest_Team', 'Closest_Distance']], 
                                       on='Team', how='left')

# Reorder the columns to place Closest Team and CTeam_Distance at the end
team_travel_data_refactored = team_travel_data_refactored[[
    'Team', 
    'Reg_Distance', 
    'Reg_Trips', 
    'Reg_AVG', 
    'N_Distance', 
    'Neutral_Site_Trips', 
    'N_AVG', 
    'Total_Distance', 
    'Overall_AVG', 
    'Closest_Team', 
    'Closest_Distance'
]]

# Drop rows with 0 regular travel distance
team_travel_data = team_travel_data_refactored[team_travel_data_refactored['Reg_Distance'] != 0]


## Find How many times each team plays their closest rival

In [ ]:
## Rename Schedule dataframe

df_schedule = schedule_data.copy()

# Remove Exhibition games from schedule
df_schedule = df_schedule[df_schedule['Conference'] != 'Exhibition']


# Match the teams in the schedule with their closest team from the travel distance data
# Extract the teams from the schedule and cross-check against the closest team

# Merging schedule data with closest team info for both home and away teams
df_schedule_merged = schedule_data.merge(
    team_travel_data[['Team', 'Closest_Team']],
    left_on='Home_Team',
    right_on='Team',
    how='left',
    suffixes=('', '_Closest_Home')
)

df_schedule_merged.rename(columns={'Closest_Team': 'Closest_Team_Home'}, inplace=True)

df_schedule_merged = df_schedule_merged.merge(
    team_travel_data[['Team', 'Closest_Team']],
    left_on='Away_Team',
    right_on='Team',
    how='left',
    suffixes=('', '_Closest_Away')
)

df_schedule_merged.rename(columns={'Closest_Team': 'Closest_Team_Away'}, inplace=True)

# Now, let's ensure both teams (home and away) are being compared properly
df_schedule_merged['Home_vs_Closest'] = df_schedule_merged['Away_Team'] == df_schedule_merged['Closest_Team_Home']
df_schedule_merged['Away_vs_Closest'] = df_schedule_merged['Home_Team'] == df_schedule_merged['Closest_Team_Away']

# Count how many times each team plays its closest opponent as either home or away
df_closest_match_count_home = df_schedule_merged.groupby('Home_Team').agg({
    'Home_vs_Closest': 'sum'
}).reset_index()

df_closest_match_count_away = df_schedule_merged.groupby('Away_Team').agg({
    'Away_vs_Closest': 'sum'
}).reset_index()

# Merge both home and away counts to ensure consistency for both teams
df_closest_match_total = pd.merge(
    df_closest_match_count_home, 
    df_closest_match_count_away, 
    left_on='Home_Team', 
    right_on='Away_Team', 
    how='outer'
)

# Replace missing values with 0 before summing up
df_closest_match_total['Home_vs_Closest'].fillna(0, inplace=True)
df_closest_match_total['Away_vs_Closest'].fillna(0, inplace=True)

# Calculate the total closest matches by summing up both columns
df_closest_match_total['Total_Closest_Matches'] = df_closest_match_total['Home_vs_Closest'] + df_closest_match_total['Away_vs_Closest']

# Rename columns for clarity and drop unneeded ones
df_closest_match_total = df_closest_match_total[['Home_Team', 'Total_Closest_Matches']].rename(columns={'Home_Team': 'Team'})

# Merge this back into the travel data
team_travel_data_final = team_travel_data.merge(
    df_closest_match_total[['Team', 'Total_Closest_Matches']],
    on='Team',
    how='left'
)

# Fill missing values with 0
team_travel_data_final['Total_Closest_Matches'].fillna(0, inplace=True)


# # Merge the schedule with the closest team data for both home and away teams
# df_schedule_merged = df_schedule.merge(team_travel_data[['Team', 'Closest_Team']], 
#                                        left_on='Home_Team', right_on='Team', how='left')

# # Now, let's check how many games each team plays against their closest team (both home and away)
# df_schedule_merged['Home_vs_Closest'] = df_schedule_merged['Away_Team'] == df_schedule_merged['Closest_Team']

# # Do the same for the away teams
# df_schedule_merged = df_schedule_merged.merge(team_travel_data[['Team', 'Closest_Team']], 
#                                               left_on='Away_Team', right_on='Team', how='left', suffixes=('_home', '_away'))

# df_schedule_merged['Away_vs_Closest'] = df_schedule_merged['Home_Team'] == df_schedule_merged['Closest_Team_away']

# # Now count how many times each team plays against their closest team, either as home or away
# df_closest_match_count = df_schedule_merged.groupby('Team_home').agg({
#     'Home_vs_Closest': 'sum',
#     'Away_vs_Closest': 'sum'
# }).reset_index()

# df_closest_match_count['Total_Closest_Matches'] = df_closest_match_count['Home_vs_Closest'] + df_closest_match_count['Away_vs_Closest']

# # Merge this back into the original travel data
# team_travel_data = team_travel_data.merge(df_closest_match_count[['Team_home', 'Total_Closest_Matches']],
#                                           left_on='Team', right_on='Team_home', how='left').drop(columns=['Team_home'])

team_travel_data_final.head()

## OUTPUT TO TEMP DIRECTORY
output_path = os.path.join('..', 'TEMP', 'team_travel_data_test_new_v3.csv')
team_travel_data_final.to_csv(output_path, index=False)


In [79]:
# team_travel_data_expanded_cleaned

## Find Each Teams Longest Trip of the Year
- and add to dataframe

In [ ]:
# For regular season games, capture Away team, Home team, and distance
away_team_info = schedule_w_distance[schedule_w_distance['Travel_Flag'] == 1][['Away_Team', 'Home_Team', 'Distance_Miles']]
away_team_info['Game_Type'] = 'Regular'

# For neutral site games, capture both teams and distance, along with the Game_Type
neutral_team_info = neutral_site_games_agg_1[['Game_ID', 'Team', 'N_Distance']]

# Create a dataframe with both Team 1 and Team 2 as opponents for neutral site games
neutral_team_info['Opponent'] = neutral_team_info['Game_ID'].str.split('_').str[2]  # Assuming the other team is in the Game_ID

# Duplicate rows to handle both teams in neutral site games
neutral_team_info_team1 = neutral_team_info[['Game_ID', 'Team', 'Opponent', 'N_Distance']].rename(columns={'N_Distance': 'Distance'})
neutral_team_info_team2 = neutral_team_info[['Game_ID', 'Opponent', 'Team', 'N_Distance']].rename(columns={'N_Distance': 'Distance', 'Opponent': 'Team', 'Team': 'Opponent'})

# Combine both team datasets for neutral site games
neutral_combined = pd.concat([neutral_team_info_team1, neutral_team_info_team2])
neutral_combined['Game_Type'] = 'Neutral'

# Regular season away games: Home_Team is the opponent
away_team_info = away_team_info.rename(columns={'Away_Team': 'Team', 'Home_Team': 'Opponent', 'Distance_Miles': 'Distance'})

# Combine both datasets (regular season and neutral site games)
combined_info = pd.concat([away_team_info[['Team', 'Opponent', 'Distance', 'Game_Type']], 
                           neutral_combined[['Team', 'Opponent', 'Distance', 'Game_Type']]])

# Clean the combined dataset by dropping rows with missing values in 'Team' or 'Distance'
combined_info_cleaned = combined_info.dropna(subset=['Team', 'Distance'])

# **Hotfix: Filter out games where the distance is 0**
combined_info_cleaned = combined_info_cleaned[combined_info_cleaned['Distance'] > 0]

## If Both Team and Opponent match drop the row
combined_info_cleaned = combined_info_cleaned[combined_info_cleaned['Team'] != combined_info_cleaned['Opponent']]
# Drop any rows with a /
combined_info_cleaned = combined_info_cleaned[~combined_info_cleaned['Opponent'].str.contains('/')]
combined_info_cleaned = combined_info_cleaned[~combined_info_cleaned['Team'].str.contains('/')]

# Now find the longest trip for each team, including the opponent and game type
longest_trip_info_cleaned = combined_info_cleaned.loc[combined_info_cleaned.groupby('Team')['Distance'].idxmax()].reset_index(drop=True)

# Merge the cleaned longest trip information back into the team_travel_data
team_travel_data_expanded_cleaned = pd.merge(team_travel_data_final, longest_trip_info_cleaned, on='Team', how='left')

# Drop the duplicate rows that were created
team_travel_data_expanded_cleaned = team_travel_data_expanded_cleaned.drop_duplicates(subset=['Team'])
# Reindex the DataFrame
team_travel_data_expanded_cleaned.reset_index(drop=True, inplace=True)

# team_travel_data_expanded_cleaned.head(20)


In [ ]:
# Sort by Distance

team_travel_data_expanded_cleaned.head(5)


### Added data clean and transform steps

In [ ]:
## Some added transformation steps
# Fill Nan Values with 0
team_travel_data_expanded_cleaned = team_travel_data_expanded_cleaned.fillna(0)

# Set Neutral Site Trips to integer
team_travel_data_expanded_cleaned['Neutral_Site_Trips'] = team_travel_data_expanded_cleaned['Neutral_Site_Trips'].astype(int)
team_travel_data_expanded_cleaned['Total_Closest_Matches'] = team_travel_data_expanded_cleaned['Total_Closest_Matches'].astype(int)

# Round the floats to 2 decimal places
team_travel_data_expanded_cleaned = team_travel_data_expanded_cleaned.round({'Reg_Distance': 2, 'Reg_Trips': 2, 'Reg_AVG': 2,
                                                                          'N_Distance': 2, 'Neutral_Site_Trips': 2, 'N_AVG': 2,
                                                                          'Total_Distance': 2, 'Overall_AVG': 2, 'Closest_Distance': 2,
                                                                          'Total_Closest_Matches': 2, 'Distance': 2})

## Rename some Columns
# Opponent to Longest_Trip_Opponent
# Distance_Longest_Trip
# Game_Type to Game_Type_Longest_Trip
team_travel_data_expanded_cleaned = team_travel_data_expanded_cleaned.rename(columns={'Opponent': 'Longest_Trip_Opponent',
                                                                                    'Distance': 'Distance_Longest_Trip',
                                                                                    'Game_Type': 'Game_Type_Longest_Trip'})

# Display The Resulting DataFrame
team_travel_data_expanded_cleaned.head(10)

# Output the Final Table to CSV

In [ ]:
# # TEMP FOLDER Output
# output_path = os.path.join('..', 'TEMP', 'FINAL_OUT_team_travel_data_v2.csv')
# team_travel_data_expanded_cleaned.to_csv(output_path, index=False)
# print(f"Output saved to: {output_path}")

import datetime
# create simple string of data (ex. 9/1/2022)
date = datetime.datetime.now().strftime("%m-%d-%Y")

# print(date)
# OUTPUT INTO DATA OUTPUT FOLDER
output_path = os.path.join('..', 'data', 'output', f'Team_Travel_Information_v2_{date}.csv')
team_travel_data_expanded_cleaned.to_csv(output_path, index=False)
print(f"Output saved to: {output_path}")

# TESTING
## Check Against Roster File
- check number of teams against number that appear in roster
- list teams missing form distance table

In [ ]:
## LOAD ROSTER DATA
roster_path = os.path.join('..', 'data', 'roster_2024_current_v3.csv')
roster_data = pd.read_csv(roster_path)

# Display the data
roster_data.head()

# get list of unique teams
teams_travel = team_travel_data_expanded_cleaned['Team'].unique() # FROM TRAVEL TABLE
teams_roster = roster_data['Current Team'].unique() # FROM ROSTER TABLE

# Print Report - length of lists and missing teams
print(f"Number of teams in travel data: {len(teams_travel)}")
print(f"Number of teams in roster data: {len(teams_roster)}")
print('----------------------------------------------------------------\n')
print("Teams in travel data but not in roster data:")
print(set(teams_travel) - set(teams_roster))
print('----------------------------------------------------------------\n')
print("Teams in roster data but not in travel data:")
print(set(teams_roster) - set(teams_travel))



missing_teams = set(teams_travel) - set(teams_roster)